In [1]:
# Set root dir
import os
os.chdir('..')
print(os.getcwd())

/Users/tonpoppe/workspace/tigger_adj_rep/tigger_adj


In [22]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import tensorflow as tf
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import yaml


In [4]:
from GAE.graph_case_controller import GraphAutoEncoder
from GAE.data_feeder_graphframes import DataFeederGraphFrames
from tigger_package.orchestrator import Orchestrator
from tigger_package.label_transfer import LabelTransferrer

In [5]:
conf = SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

23/12/29 21:30:04 WARN Utils: Your hostname, MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 10.100.81.74 instead (on interface en0)
23/12/29 21:30:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/29 21:30:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


23/12/29 21:30:20 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
config_dict = yaml.safe_load('''
    weight_col: 'edge_att0'
    label_col: none
    support_size: [4, 4]
    dims: [2, 6, 6, 6]
    batch_size: 512
    hub0_dim: 6
    seed: 1
    learning_rate: 0.001
    usebn: true
    verbose: true
''')

In [19]:
folder = "data/erdos/"
orchestrator = Orchestrator(folder)
nodes = orchestrator._load_nodes()
edges = orchestrator._load_edges()                                 

/Users/tonpoppe/anaconda3/envs/tg_geo/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


The following node labels are included ['attr0', 'attr1', 'attr2', 'attr3', 'attr4', 'attr5', 'attr6', 'attr7']


The following edge labels are included ['weight', 'edge_att1', 'edge_att2', 'edge_att3', 'edge_att4', 'edge_att5', 'edge_att6']


train nodes [399 386 575 684 163  43 278 616 132 774] ...
val nodes [312 980 209  55 342  49 707 687 857 669] ...


In [ ]:
ga